In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Feb 23 10:59:46 2020

https://dash.plotly.com/

@author: Christo Strydom
"""
import tensorflow as tf
from tensorflow import keras

import os
import tempfile
import datetime as datetime
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import pickle
import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
os.chdir('/media/lnr-ai/christo/github_repos/ae_mp/')
# from sklearn.model_selection import train_test_split
# import pandas as pd
# import numpy as np
#%%
mpl.rcParams['figure.figsize'] = (12, 10)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [2]:
%matplotlib inline

In [3]:
ae_data_path='/media/lnr-ai/christo/github_repos/ae_mp/data/'
ae_candles_filename = 'data/USDZAR_Candlestick_5_M_BID_01.01.2019-08.02.2020_wrangled.csv'
ae_df=pd.read_csv(ae_candles_filename)
ae_df=ae_df[ae_df.is_business_day].copy()

In [4]:
def pkl_dump(df, fname, data_path):
   f = open("{data_path}{fname}.pkl".format(fname=fname,data_path=data_path),"wb")
   pickle.dump(df,f)
   f.close()
   return

def pkl_load(fname, data_path):
   return pickle.load( open( "{data_path}{fname}.pkl".format(fname=fname, data_path=data_path), "rb" ) )  

def time_filter_fn(df, start_int_seconds, end_int_seconds):
   return (ae_df['int_seconds']>=start_int_seconds) & (ae_df['int_seconds']<=end_int_seconds)

def businessday_filter_fn(df):
   return df['is_business_day']

def mp_filter_fn(df, start_int_seconds, end_int_seconds):
   time_filter=time_filter_fn(df, start_int_seconds, end_int_seconds)
   businessday_filter=businessday_filter_fn(df)
   return time_filter&businessday_filter

def days_fn(df,mp_filter):
   return list(set(df.loc[mp_filter, 'date'])) 
# days=list(set(ae_df.loc[mp_filter, 'date'])) 

def day_filter_fn(df,day):
   return df['date']==day

def range_fn(df, day_filter,time_filter):
    f=np.logical_and(day_filter, time_filter)
    on_high=max(df.loc[f, 'high'])
    on_low=min(df.loc[f, 'low'])   
    on_range=on_high-on_low
    return on_high, on_low, on_range

def on_fn(df, day_filter, on_high, on_low, on_range):
   df.loc[day_filter, 'on_low']=on_low
   df.loc[day_filter, 'on_high']=on_high
   df.loc[day_filter, 'on_range']=on_range
   return df

def order_fn(df,day_filter, perc_level, on_high, on_low, on_range):   
   df.loc[day_filter, 'sell_{perc_level}'.format(perc_level=perc_level)]=on_high+perc_level*on_range/100
   df.loc[day_filter, 'buy_{perc_level}'.format(perc_level=perc_level)]=on_low-perc_level*on_range/100
   return df

def trade_range_fn(df, day):
    day_filter=day_filter_fn(df, day)
    time_filter=time_filter_fn(df, 0, 28800)
    on_high, on_low, on_range=range_fn(df, day_filter, time_filter)
    df=on_fn(df, day_filter, on_high, on_low, on_range)
    for perc_level in [50,75,100]:
        df=order_fn(df=df,day_filter=day_filter, 
                  perc_level=perc_level, 
                  on_high=on_high, 
                  on_low=on_low, 
                  on_range=on_range)
    return df

def trade_df_fn(df, dfilter, tfilter):
    return df[np.logical_and(dfilter, tfilter)]
# on_high, on_low, on_range=range_fn(df=ae_df, day_filter=day_filter, time_filter=time_filter)

In [5]:
#%%
# ae_df['jhb_timestamp']==2019-01-01 02:05:00
list(ae_df)
ae_df_list=['datetime',
 'date',
 'Gmt time',
 'Open',
 'High',
 'Low',
 'Close',
 'Volume',
 'bidopen',
 'bidclose',
 'bidhigh',
 'bidlow',
 'askopen',
 'askclose',
 'askhigh',
 'asklow',
 'tickqty',
 'open',
 'high',
 'low',
 'close',
 'period',
 'dom',
 'awdn',
 'month',
 'doy',
 'wny',
 'timestamp',
 'ny_timestamp',
 'london_timestamp',
 'jhb_timestamp',
 'int_seconds',
 'universaldate',
 'is_business_day',
 'previous_business_day',
 'five_previous_business_day']
format_str=MP_FXCM_DATETIME_FORMAT='%Y-%m-%d %H:%M:%S'
ae_df['jhb_timestamp'] =  pd.to_datetime(ae_df['jhb_timestamp'], format=format_str)
format_str='%Y-%m-%d'
ae_df['date'] =  pd.to_datetime(ae_df['date'], format=format_str)
# ae_df['overnight_range']=False
# time_filter=(ae_df['int_seconds']>=0) & (ae_df['int_seconds']<=28800)
# businessday_filter=ae_df['is_business_day']
# mp_filter=time_filter&businessday_filter

In [ ]:
mp_filter=mp_filter_fn(df=ae_df, start_int_seconds=0, end_int_seconds=28800)
ae_df['overnight_range']=mp_filter
days = days_fn(ae_df,mp_filter)
for day in days:
   index=days.index(day)
   print('Doing day = {day} and index = {index}...'.format(index=index, day=day))
   ae_df=trade_range_fn(ae_df, day)

In [ ]:
pkl_dump(df=ae_df, fname='ae_df.csv', data_path=ae_data_path)

In [6]:
ae_df=pkl_load(fname='ae_df.csv', data_path=ae_data_path)
ae_df['date'] =  pd.to_datetime(ae_df['date'], format=format_str)
ae_df=ae_df[ae_df.is_business_day].copy()
day=max(ae_df['date'])

In [7]:
day_filter=day_filter_fn(ae_df, day)

In [8]:
df=ae_df[day_filter].copy()

In [9]:
day

Timestamp('2020-02-07 00:00:00')

In [10]:
time_filter=time_filter_fn(ae_df, 0, 28800)

In [11]:
len(np.where(day_filter)[0])

288

In [12]:
on_high, on_low, on_range=range_fn(df=ae_df, day_filter=day_filter, time_filter=time_filter)

In [13]:
df=on_fn(df, day_filter, on_high, on_low, on_range)

In [14]:
for perc_level in [50,75,100]:
    df=order_fn(df=df,day_filter=day_filter, 
              perc_level=perc_level, 
              on_high=on_high, 
              on_low=on_low, 
              on_range=on_range)

In [15]:
dst=list(df['datetime'])[-1]
# dst.year
date_object = datetime.datetime.strptime(dst, "%Y-%m-%d %H:%M:%S")

In [16]:
date_object.year

2020

In [17]:
d=datetime.datetime(date_object.year, date_object.month,date_object.day, 8, 30)

In [18]:
d.strftime("%Y-%m-%d %H:%M:%S")

'2020-02-07 08:30:00'

In [19]:
def time_fn(dst):
    date_object = datetime.strptime(dst, "%Y-%m-%d %H:%M:%S")
    end_d=datetime(date_object.year, date_object.month,date_object.day, 8,0)
    start_d=datetime(date_object.year, date_object.month,date_object.day, 0, 0)
    fin_d=datetime(date_object.year, date_object.month,date_object.day, 23,59,59)
    return start_d.strftime("%Y-%m-%d %H:%M:%S"), end_d.strftime("%Y-%m-%d %H:%M:%S"), fin_d.strftime("%Y-%m-%d %H:%M:%S")

In [48]:
import plotly.graph_objects as go

import pandas as pd
from datetime import datetime

# df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/finance-charts-apple.csv')
df=ae_df[day_filter].copy()
min_y=min(min(df['Low']),min(df.buy_100))*0.995
max_y=max(max(df['High']),max(df.sell_100))*1.005
fig = go.Figure(data=[go.Candlestick(x=df['datetime'],
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'])])
start_time, end_time, fin_time = time_fn(dst=list(df['datetime'])[0])
# fig.update_xaxes(range=[min(df.buy_100), max(df.sell_100)])
# fig.update_yaxes(range=[start_time, end_time])


# start_time=list(df['datetime'])[0]
# end_time=list(df['datetime'])[-1]

fig.update_layout(
    title='Overnight trade',
    yaxis_title='USDZAR',
    shapes = [
        dict(
#         type="rect",
        x0=end_time, x1=fin_time, y0=list(df.sell_100)[0], y1=list(df.sell_100)[0],
        line=dict(color="red",width=1,dash="dashdot"),opacity=0.2),
        dict(
        x0=end_time, x1=fin_time, y0=list(df.sell_75)[0], y1=list(df.sell_75)[0],
        line=dict(color="red",width=1,dash="dashdot"),opacity=0.3),
        dict(
        x0=end_time, x1=fin_time, y0=list(df.sell_50)[0], y1=list(df.sell_50)[0],
        line=dict(color="red",width=1,dash="dashdot"),opacity=0.4),
        dict(
        x0=end_time, x1=fin_time, y0=list(df.buy_100)[0], y1=list(df.buy_100)[0],
        line=dict(color="green",width=1,dash="dashdot"),opacity=0.2),
        dict(
        x0=end_time, x1=fin_time, y0=list(df.buy_75)[0], y1=list(df.buy_75)[0],
        line=dict(color="green",width=1,dash="dashdot"),opacity=0.3),
        dict(
        x0=end_time, x1=fin_time, y0=list(df.buy_50)[0], y1=list(df.buy_50)[0],
        line=dict(color="green",width=1,dash="dashdot"),opacity=0.4),      
        dict(
        type="rect",
        x0=start_time, x1=end_time, y0=list(df.on_low)[0], y1=list(df.on_high)[0],
        fillcolor="LightSkyBlue", opacity=0.15),
        dict(
        type="rect",
        x0=start_time, x1=end_time, y0=min_y, y1=max_y,
        fillcolor="LightSkyBlue", opacity=0.05),        
#         dict(
#         x0=start_time, x1=end_time, y0=list(df.on_low)[0], y1=list(df.on_low)[0],
#         line_width=1),
#         dict(
#         x0=start_time, x1=end_time, y0=list(df.on_high)[0], y1=list(df.on_high)[0],
#         line_width=1)
    ])

fig.add_trace(go.Scatter(
    x=[start_time,end_time],
    y=[15,15],
    name="yaxis1 data"
))
# fig.update_xaxes(range=[min(df.buy_100), max(df.sell_100)])
# fig.update_yaxes(range=[start_time, end_time])
# fig.add_shape(
#         # filled Rectangle
#             type="rect",
#             xref="x",
#             yref="y",    
#             x0=start_time,
#             y0=df.buy_100,
#             x1=end_time,
#             y1=df.sell_100,
#             line=dict(
#                 color="RoyalBlue",
#                 width=2,
#             ),
#             fillcolor="LightSkyBlue",
#         )
# fig.add_shape(
#         # Rectangle reference to the plot
#             type="rect",
#             xref="paper",
#             yref="paper",
#             x0=0.25,
#             y0=0,
#             x1=0.5,
#             y1=0.5,
#             line=dict(
#                 color="LightSeaGreen",
#                 width=3,
#             ),
#             fillcolor="PaleTurquoise",
#         )

# fig.add_shape(
#         # Line reference to the axes
#             type="line",
#             xref="x",
#             yref="y",
#             x0=4,
#             y0=0,
#             x1=8,
#             y1=1,
#             line=dict(
#                 color="LightSeaGreen",
#                 width=3,
#             ),
#         )
# fig.update_yaxes(range=[min_x, max_x])
#     shapes = [dict(
#         x0='2019-08-21 00:00:00', x1='2019-08-21 23:55:00', y0=list(df.on_high)[0], y1=list(df.on_high)[0],
#         line_width=1)],
# fig.update_layout(
#     title='The Great Recession',
#     yaxis_title='USDZAR',
# #     shapes = [dict(
# #         x0='2019-08-21 00:00:00', x1='2019-08-21 23:55:00', y0=list(df.on_low)[0], y1=list(df.on_low)[0],
# #         line_width=1)],
    
#     shapes = [dict(
#         x0='2019-08-21 00:00:00', x1='2019-08-21 23:55:00', y0=list(df.on_high)[0], y1=list(df.on_high)[0],
#         line_width=1)])
    
#     annotations=[dict(
#         x='2016-12-09', y=0.05, xref='x', yref='paper',
#         showarrow=False, xanchor='left', text='Increase Period Begins')]


fig.show()

In [23]:
def trade_df_fn(df, dfilter, tfilter):
    return df[np.logical_and(dfilter, tfilter)]

In [45]:
tfilter=time_filter_fn(ae_df.copy(), 28800, 24*60*60)
trade_df=trade_df_fn(df=ae_df.copy(), dfilter=day_filter, tfilter=tfilter)
trade_df['mtm']=0
trade_df['pos']=0
trade_df['trade']=0
trade_df['entry_price']=0
strade_dict=dict()
spos=0
trade_df_maxindex=max(trade_df.index)
for index, (trade_df_index,dt,close,sell_50,sell_75,sell_100) in enumerate(zip(trade_df.index,trade_df['datetime'],trade_df['Close'], trade_df['sell_50'],trade_df['sell_75'],trade_df['sell_100'])):
    print(index,close,dt)
    if (close>sell_50) and (trade_df.loc[trade_df_index-1,'pos']==0):
#         spos+=1
#         trade_df.loc[range(trade_df_index,trade_df_maxindex+1),'spos']=spos
        trade_df.loc[trade_df_index,'trade']=-1
        trade_df.loc[trade_df_index,'entry_price']=close
        average_entry_price=trade_df['trade']
    if (close>sell_75) and (trade_df.loc[trade_df_index-1,'pos']==-1):
        spos+=1                
#         trade_df.loc[range(trade_df_index,trade_df_maxindex+1),'spos']=spos
        trade_df.loc[trade_df_index,'trade']=-1
        trade_df.loc[trade_df_index,'entry_price']=close    
    if (close>sell_100) and (trade_df.loc[trade_df_index-1,'pos']==-2):
        spos+=1                
#         trade_df.loc[range(trade_df_index,trade_df_maxindex+1),'spos']=spos
        trade_df.loc[trade_df_index,'trade']=-1
        trade_df.loc[trade_df_index,'entry_price']=close    
    trade_df['pos']=trade_df['trade'].cumsum()
    trade_df.loc[trade_df_index,'mtm']=sum(trade_df['trade'].values*close- np.multiply(trade_df['trade'], trade_df['entry_price']))

0 14.98673 2020-02-07 08:00:00
1 14.98126 2020-02-07 08:05:00
2 14.9938 2020-02-07 08:10:00
3 15.00056 2020-02-07 08:15:00
4 15.001220000000002 2020-02-07 08:20:00
5 14.989529999999998 2020-02-07 08:25:00
6 14.98913 2020-02-07 08:30:00
7 14.97393 2020-02-07 08:35:00
8 14.96996 2020-02-07 08:40:00
9 14.972129999999998 2020-02-07 08:45:00
10 14.98313 2020-02-07 08:50:00
11 14.98731 2020-02-07 08:55:00
12 14.992729999999998 2020-02-07 09:00:00
13 14.98833 2020-02-07 09:05:00
14 14.996529999999998 2020-02-07 09:10:00
15 14.992820000000002 2020-02-07 09:15:00
16 14.98543 2020-02-07 09:20:00
17 15.01233 2020-02-07 09:25:00
18 15.00653 2020-02-07 09:30:00
19 15.020829999999998 2020-02-07 09:35:00
20 15.02546 2020-02-07 09:40:00
21 15.05332 2020-02-07 09:45:00
22 15.03173 2020-02-07 09:50:00
23 15.03043 2020-02-07 09:55:00
24 15.0229 2020-02-07 10:00:00
25 15.00693 2020-02-07 10:05:00
26 15.03231 2020-02-07 10:10:00
27 15.02303 2020-02-07 10:15:00
28 15.01153 2020-02-07 10:20:00
29 15.01782999

In [25]:
trade_df['entry_price']=0
trade_df.loc[trade_df['trade']!=0,'entry_price']=trade_df[trade_df['trade']!=0]['close']

In [26]:
trade_df_index

112672

In [35]:
np.dot(trade_df['trade'], trade_df['entry_price'])/trade_df['pos'].values[-1]

15.08778

In [34]:
trade_df['pos'].values[-1]

-3

In [36]:
close

15.05669

In [38]:
(np.dot(trade_df['trade'], trade_df['entry_price'])/trade_df['pos'].values[-1]-close)*trade_df['pos'].values[-1]

-0.09327000000000218

In [39]:
trade_df['trade']*-1

112481    0
112482    0
112483    0
112484    0
112485    0
         ..
112668    0
112669    0
112670    0
112671    0
112672    0
Name: trade, Length: 192, dtype: int64

In [42]:
sum(trade_df['trade'].values*close- np.multiply(trade_df['trade'], trade_df['entry_price']))

0.09327000000000041

In [46]:
trade_df['mtm']

112481    0.00000
112482    0.00000
112483    0.00000
112484    0.00000
112485    0.00000
           ...   
112668    0.09327
112669    0.09327
112670    0.09327
112671    0.09327
112672    0.09327
Name: mtm, Length: 192, dtype: float64